### Load and analyze the dataset

In [1]:
from datasets import load_dataset

# Load the custom dataset (or it if it does not exist)
# Param 1: location of the dataset loader script
# Param 2: location of cache folder, where the dataset will be saved
dataset = load_dataset(r'C:\Users\Habram\Documents\thesis-masters\Manual_annotation\SER_dataset.py',
                       cache_dir=r'C:\Users\Habram\.cache')

c:\Users\Habram\Anaconda3\envs\LayoutXLM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset ser_dataset downloaded and prepared to C:/Users/Habram/.cache/ser_dataset/default/0.0.0/b2751c2f0cd04c2898ea55bb5e5b9596c8284b0ccbdea3a84c8e60afc0a18719. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 220.81it/s]


In [2]:
dataset

DatasetDict({
    test: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
        num_rows: 10
    })
})

In [3]:
features = dataset["test"].features
label_column_name = "ner_tags"

# Define the dictionaries which associate the labels with integer IDs
label_list = features[label_column_name].feature.names
id2label = {k: v for k,v in enumerate(label_list)}
label2id = {v: k for k,v in enumerate(label_list)}

id2label

{0: 'O',
 1: 'B-R_NAME',
 2: 'I-R_NAME',
 3: 'B-R_STREET',
 4: 'I-R_STREET',
 5: 'B-R_HOUSENUMBER',
 6: 'I-R_HOUSENUMBER',
 7: 'B-R_ZIP',
 8: 'I-R_ZIP',
 9: 'B-R_CITY',
 10: 'I-R_CITY',
 11: 'B-R_COUNTRY',
 12: 'I-R_COUNTRY',
 13: 'B-S_NAME',
 14: 'I-S_NAME',
 15: 'B-S_STREET',
 16: 'I-S_STREET',
 17: 'B-S_HOUSENUMBER',
 18: 'I-S_HOUSENUMBER',
 19: 'B-S_ZIP',
 20: 'I-S_ZIP',
 21: 'B-S_CITY',
 22: 'I-S_CITY',
 23: 'B-S_COUNTRY',
 24: 'I-S_COUNTRY',
 25: 'B-S_BANK',
 26: 'I-S_BANK',
 27: 'B-S_IBAN',
 28: 'I-S_IBAN',
 29: 'B-I_NUMBER',
 30: 'I-I_NUMBER',
 31: 'B-I_DATE',
 32: 'I-I_DATE',
 33: 'B-I_AMOUNT',
 34: 'I-I_AMOUNT'}

### Load the model, tokenizer, image processor

In [4]:
from transformers import LayoutLMv2ImageProcessor, LayoutXLMTokenizer
from transformers import LayoutLMv2Model
import torch

image_processor = LayoutLMv2ImageProcessor(apply_ocr=False)
tokenizer = LayoutXLMTokenizer.from_pretrained("microsoft/layoutxlm-base")

# Load the pre-trained model without classification head
model = LayoutLMv2Model.from_pretrained("microsoft/layoutxlm-base")

Some weights of the model checkpoint at microsoft/layoutxlm-base were not used when initializing LayoutLMv2Model: ['layoutlmv2.visual.backbone.bottom_up.res4.18.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.1.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.16.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.4.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.19.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.16.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.1.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.0.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.9.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.13.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.10.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.ba

### Extract the embeddings (output of the LayoutXLM model)

In [5]:
from dataclasses import dataclass
from torch.utils.data import DataLoader

@dataclass
class DataCollatorForTokenClassification:
    feature_extractor: LayoutLMv2ImageProcessor
    tokenizer: LayoutXLMTokenizer

    def __call__(self, examples):
        tokens = []
        bboxes = []
        ner_tags = []
        images = []

        for ex in examples:
            images.append(ex['image'])

        img_features = image_processor(images, return_tensors='pt').pixel_values

        for ex in examples:
            tokens.append(ex['tokens'])
            bboxes.append(ex['bboxes'])
            ner_tags.append(ex['ner_tags'])

        encoding = tokenizer(tokens, 
                     boxes=bboxes, 
                     word_labels=ner_tags, 
                     truncation=True, 
                     padding="max_length", 
                     max_length=512,
                     return_tensors='pt'
                     )

        encoding['image'] = img_features

        return encoding
    
data_collator = DataCollatorForTokenClassification(
    image_processor,
    tokenizer,
)

# Either 'train' or 'test' set, modify if needed!
dataloader = DataLoader(dataset['test'], 
                        batch_size=1, 
                        collate_fn=data_collator)

In [6]:
import numpy as np

def group_subword_embeddings(tokens, embeddings):
    groups = []
    current_group = []
    for idx, token in enumerate(tokens):
        # !Not an underscore! _ ▁, but a special character
        if token.startswith('▁'):
            if len(current_group) > 0:
                groups.append(current_group)
            current_group = [idx]
        elif token == '<s>' or token == '</s>' or token == '<pad>':
            continue
        else:
            current_group.append(idx)
    groups.append(current_group)

    grouped_embeddings = []
    grouped_tokens = []
    for group in groups:
        grouped_tokens.append(''.join([tokens[i].lstrip('▁') for i in group]))
        grouped_embeddings.append(np.array(embeddings[group[0]: group[-1]+1].mean(axis=0)))

    return grouped_tokens, np.array(grouped_embeddings)

In [7]:
from tqdm import tqdm

size_of_dataset = 10

iterator = iter(dataloader)

all_labels = []
all_embeddings = []
all_tokens = []

# Iterate over the dataset
for iteration in tqdm(range(size_of_dataset)):
    # Preprocess one example
    example = next(iterator)

    # Remove the labels from the example + remove the -100 elements
    labels = example.pop('labels')
    labels = np.array(labels)
    labels = np.delete(labels, np.where(labels == -100))
    labels = torch.tensor(labels)

    # Perform a forward-pass on the XLM model
    with torch.no_grad():
        output = model(**example, output_hidden_states=True)

    # Get the output embeddings of the XLM model
    xlm_embeddings = output['last_hidden_state'][0]

    # Get the input IDs
    input_ids = example["input_ids"]
    
    # Convert the input IDs to sub-words
    tokens = tokenizer.convert_ids_to_tokens(*input_ids)

    # Group the sub-words and embeddings
    joined_tokens, joined_embeddings = group_subword_embeddings(tokens, xlm_embeddings)

    all_labels.append(labels)
    all_embeddings.append(joined_embeddings)
    all_tokens.append(joined_tokens)

100%|██████████| 10/10 [00:17<00:00,  1.71s/it]


In [8]:
# Save the embeddings, labels and tokens

train_data = {
    'embeddings': all_embeddings,
    'tokens':     all_tokens,
    'labels':     all_labels
}

np.save('test_data.npy', np.array(train_data, dtype=object), allow_pickle=True)

### Load the classifier

In [9]:
import torch.nn as nn
import torch.optim as optim
import torch

softmax = nn.Softmax(dim=1)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(768, 35)

    def forward(self, x):
        x = self.fc(x)
        return x

classifier = Net()
classifier.load_state_dict(torch.load(r'C:\Users\Habram\Documents\thesis-masters\LayouXLM\Custom\classifier.pt'))

<All keys matched successfully>

In [10]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class IstvoicesDataset(Dataset):
    def __init__(self, data):
        self.labels = data['labels']
        self.embeddings =  data['embeddings']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        emb = self.embeddings[idx]
        lab = self.labels[idx]

        return emb, lab

testing_data = np.load('test_data.npy', allow_pickle=True).item()
dataset_test_torch = IstvoicesDataset(testing_data)
test_dataloader = DataLoader(dataset_test_torch, batch_size=1, shuffle=True)

### Evaluate

In [11]:
def iob_to_label(label):
    label = label[2:]
    if not label:
      return 'other'
    return label.lower()

def get_all_predictions():
    all_labels = []
    all_predictions = []
    
    # iterate over the whole test set
    for test_idx in range(len(testing_data['embeddings'])):
        test_features, test_labels = next(iter(test_dataloader))
        predictions = classifier(test_features)
        predictions = softmax(predictions[0])
        predictions = torch.argmax(predictions, dim=1)
        predictions = predictions.numpy()
        test_labels = test_labels[0].numpy()

        # Get the actual labels (str)
        true_predictions = [id2label[pred] for pred, label in zip(predictions, test_labels)]
        true_labels      = [id2label[label] for pred, label in zip(predictions, test_labels)]

        all_labels.append(list(map(iob_to_label, true_labels)))
        all_predictions.append(list(map(iob_to_label, true_predictions)))

    return all_labels, all_predictions

all_labels, all_predictions = get_all_predictions()

all_labels = [item.lower() for sublist in all_labels for item in sublist]
all_predictions = [item.lower() for sublist in all_predictions for item in sublist]

In [12]:
label_list = ['other',
              'r_name', 'r_street', 'r_housenumber', 'r_zip', 'r_city', 'r_country',
              's_name', 's_street', 's_housenumber', 's_zip', 's_city', 's_country',
              's_bank', 's_iban','i_number','i_date', 'i_amount',]

In [13]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15, 15))
cm = confusion_matrix(y_true=all_labels, y_pred=all_predictions, labels=label_list, normalize='pred')
cm_display = ConfusionMatrixDisplay(cm, display_labels=label_list)
cm_display.plot(ax=ax, xticks_rotation='vertical')
plt.title('Confusion matrix of LayoutXLM on IstVoices_de (all document)')
plt.show()

### Visualize a prediction

In [20]:
# Get a raw example from the test set
example = dataset['test'][5]

# Prepare the example for the model
encoding = tokenizer(example['tokens'], boxes=example['bboxes'], word_labels=example['ner_tags'],
                     truncation=True, padding="max_length", max_length=512, return_tensors='pt')

img_encoding = image_processor(example['image'], return_tensors='pt').pixel_values

encoding['image'] = img_encoding

# Remove the labels from the encoding
labels = encoding.pop('labels')
labels = np.array(labels)
labels = np.delete(labels, np.where(labels == -100))
labels = torch.tensor(labels)

# Send the example through the model
with torch.no_grad():
    output = model(**encoding)

# Get the output embeddings of the XLM model
xlm_embeddings = output['last_hidden_state'][0]

# Get the input IDs
input_ids = encoding["input_ids"]

# Convert the input IDs to sub-words
tokens = tokenizer.convert_ids_to_tokens(*input_ids)

# Group the sub-words and embeddings
joined_tokens, joined_embeddings = group_subword_embeddings(tokens, xlm_embeddings)

# Send the embeddings through the classifier
predictions = classifier(torch.tensor(joined_embeddings))
predictions = softmax(predictions)
predictions = torch.argmax(predictions, dim=1)
predictions = predictions.numpy()
labels = labels.numpy()

# Get the actual labels (str) instead of IDs
true_predictions = [id2label[pred] for pred, label in zip(predictions, labels)]
true_labels      = [id2label[label] for pred, label in zip(predictions, labels)]

true_predictions = list(map(iob_to_label, true_predictions))
true_labels = list(map(iob_to_label, true_labels))

print(true_predictions)
print(true_labels)

['other', 'other', 'r_name', 'r_name', 'r_name', 's_name', 'r_city', 'r_street', 'r_housenumber', 'r_zip', 'r_city', 's_name', 'r_city', 's_name', 'r_name', 'r_name', 'other', 'other', 's_name', 's_street', 's_housenumber', 'r_zip', 's_city', 'other', 'other', 'r_country', 's_iban', 'other', 'other', 'i_number', 'i_date', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'i_date', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 'other', 

In [21]:
from PIL import ImageDraw, ImageFont

draw = ImageDraw.Draw(example['image'])

font = ImageFont.load_default()

def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

label2color = {
        'r_name':           'lightcoral',
        'r_street':         'brown',
        'r_housenumber':    'red', 
        'r_zip':            'Salmon',
        'r_city':           'chocolate',
        'r_country':        'Sandybrown',
        's_name':           'olive',      
        's_street':         'yellowgreen',
        's_housenumber':    'lawngreen',
        's_zip':            'palegreen',
        's_city':           'forestgreen',
        's_country':        'limegreen',
        's_bank':           'darkgreen',
        's_iban':           'teal',
        'i_number':         'aqua',
        'i_date':           'deepskyblue',
        'i_amount':         'blue',
        'other':            'magenta'
    }

width, height = example['image'].size

for prediction, box in zip(true_predictions, example['bboxes']):
    box_normalized = unnormalize_box(box, width, height)
    # Draw the label bounding box and name on the invoice
    draw.rectangle(box_normalized, outline=label2color[prediction])
    draw.text((box_normalized[0] + 10, box_normalized[1] - 10), text=prediction, fill=label2color[prediction], font=font)

example['image'].show()